In [7]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

plt.style.use("seaborn-darkgrid")

In [6]:
# setup path to sumout files
sumout_path = "../pydata/sumout_files/"
sumout_files = glob.glob(sumout_path+"*.sumout")
print(sumout_files)

['../pydata/sumout_files/BD+12_1917.sumout', '../pydata/sumout_files/BD+12_1926.sumout']


In [161]:
# read the sumout file into a pandas dataframe
# this is tricky as the sumout files are subdivided into individual tables for each species

with open(sumout_files[0]) as sumout_file:
    species_dict = {}
    lines = sumout_file.readlines()
    stellar_info = lines[0:3]
    lines = lines[4:]
    #mask out some lines that aren't needed
    mask_list = ["E.P.", "R.W.", "wavelength trends", "wav. correl.", "average abundance"]
    line_mask = [not any(x in line for x in mask_list) for line in lines]
    masked_lines = np.array(lines)[line_mask]
    
    #find the gaps between species tables
    species_gaps = np.where(masked_lines=='\n')[0]
    for idx,gap in enumerate(species_gaps):
        if idx == 0:
            last_gap = 0
        else:
            last_gap = species_gaps[idx-1]
        species_lines = masked_lines[last_gap:gap]
        
        if species_lines[0] == "\n":
            species_lines = species_lines[1:]
            
        species = species_lines[0][30:35].replace(' ','')
        input_abund = species_lines[0][-7:-2]
        
        # read this to a tmp file
        with open("tmp.txt", "w") as out:
            for line in species_lines:
                if line[0:2] == "  ":
                    out.write(line[2:])
                else:
                    out.write(line)
        # read the tmp file out to a pandas dataframe
        df = pd.read_csv("tmp.txt", delim_whitespace=True,skiprows=1)
        species_dict[species] = (input_abund, df)
            
     